<a href="https://colab.research.google.com/github/DogaSahin/Twitter_Sentiment_Analysis/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweepy
!pip install textblob
!pip install preprocessor
!pip install statistics

  Created wheel for preprocessor: filename=preprocessor-1.1.3-cp37-none-any.whl size=4477 sha256=0b56d4e19327c449a2b469bdb000f43f34666108b44760b9043f8e27df844fe1
  Stored in directory: /root/.cache/pip/wheels/98/c1/a2/21fbcfd80d76576bbf148991a66f00730f541f265c7600000f
Successfully built preprocessor
  Created wheel for statistics: filename=statistics-1.0.3.5-cp37-none-any.whl size=7454 sha256=818b7857cc7be76180a6faad7829d76c581dbf2dc0bc913236d480cb74311a06
  Stored in directory: /root/.cache/pip/wheels/75/55/90/73aa7662bfb4565b567618547a275f01372a678ca92ecd64f3
Successfully built statistics


Getting Tweets from twitter with desired keywords


In [ ]:
import tweepy as tw
import textblob
import preprocessor
import statistics
import numpy as np
import pandas as pd
from typing import List

In [ ]:
api_key = "zs1VrItXow1kXKZhNtnbn8OQD"
api_secret_key = "CW47HJK9SnYjJTNVsMMdCfyjK0x3N8hKFTAXCSaTbpmqBMMMMI"
access_token = "1206982141172764672-3hFfVWIWYzz8dEuJnGnszPdhGY5FEE"
access_token_secret = "X7IXHwBdXKAix3BdmlZ9mhWSxXfB9OPFQwGstT7dSIMaa"
Bearer_token: "AAAAAAAAAAAAAAAAAAAAAMYGNwEAAAAAOcxExg4tCZGNHJ9tiUlvK3FsM58%3DOPbPEUBUDSag04nkiHSf8JWAdts7qkC34GUgeFNKrRXDk2MDMn"

auth = tw.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


In [ ]:
def get_tweets(keyword: str)->List[str]:
  all_tweets = []
  for tweet in tw.Cursor(api.search, q = keyword, tweet_mode = 'extended',lang='en').items(10):
    all_tweets.append(tweet.full_text)

  return all_tweets

def cleaning_tweets(all_tweets:List[str])->List[str]:
  clean_tweets=[]
  for tweet in all_tweets:
    clean_tweets.append(p.clean(tweet))

  return clean_tweets

keywords = ["covid","coronavirus","covid19"]
get_tweets(keywords)


['Updated: @KFF brief on global funding across the six #COVID19 emergency supplemental bills. \n\nThe most recent bill, the American Rescue Plan Act, includes the most funding for the global response to date.\n\nhttps://t.co/tzVn2TjG8O',
 'RT @ROWPublicHealth: Keep practicing public health measures. COVID-19 spreads easily between loved ones and friends, those with whom we may…',
 "Better late than never! SMH\n\nCDC's new school distancing guidelines say 3 feet apart is often enough\n\nhttps://t.co/uhHSlW4eUe  #coronavirus #COVID19 #vaccines #schoolreopening",
 'Swift claims settlement is key to rebuilding Covid-hit reputation says legal alliance @gilawconnect\n\nhttps://t.co/IF7nlKppXC\n\n#insurance #pandemic #coronavirus #covid19 #claims #claimspayouts #businessinterruption #reputation',
 'RT @CPHO_Canada: Today’s #COVID19 in 🇨🇦 info and updates:\n➡️#GOC website on #coronavirus disease: \nhttps://t.co/8kWHvr0h7S\n➡️Canada COVID-1…',
 'The @WMO has warned that #weather &amp; #climate 

Training model from labeled twitter data


In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
from matplotlib import rcParams
from collections import Counter
from nltk.tokenize import RegexpTokenizer
import re
import string

import warnings
warnings.filterwarnings("ignore")

In [4]:
data = pd.read_csv("/content/training.1600000.processed.noemoticon.csv", encoding = "ISO-8859-1", engine="python")
data.columns = ["label", "time", "date", "query", "username", "text"]
data.head()


,label,time,date,query,username,text
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599998,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [5]:
data.columns
print(len(data))
data.shape
data.info()
data.dtypes


1599999
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   label     1599999 non-null  int64 
 1   time      1599999 non-null  int64 
 2   date      1599999 non-null  object
 3   query     1599999 non-null  object
 4   username  1599999 non-null  object
 5   text      1599999 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


label        int64
time         int64
date        object
query       object
username    object
text        object
dtype: object